In [1]:
!pip install pyspark

     |████████████████████████████████| 204.7MB 64kB/s 
     |████████████████████████████████| 204kB 38.6MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.0-py2.py3-none-any.whl size=205044182 sha256=35283ca4d701a229103f807fdbdf7801af4d0b0f12955c7d01ebe7cab5241d42
  Stored in directory: /root/.cache/pip/wheels/57/27/4d/ddacf7143f8d5b76c45c61ee2e43d9f8492fc5a8e78ebd7d37
Successfully built pyspark


In [ ]:
Date, Location , Total Sales



In [2]:
import datetime
import random

def generate_random_records():
  start_date = datetime.date(2020, 1, 1)
  end_date = datetime.date(2020, 6, 1)
  time_between_dates = end_date - start_date
  days_between_dates = time_between_dates.days
  random_number_of_days = random.randrange(days_between_dates)
  random_date = start_date + datetime.timedelta(days=random_number_of_days)
  location = ['Mumbai','Delhi','Kolkata','Banglore']
  random_location = random.choice(location)
  random_integer = random.randint(2000,10000)
  return [random_date,random_location,random_integer]
print(generate_random_records())

[datetime.date(2020, 2, 20), 'Mumbai', 5314]


In [18]:
import pandas as pd
data = pd.DataFrame(columns = ('date','location','sales'))

In [19]:
for i in range(6000):
  data.loc[i] = generate_random_records()

In [20]:
data.head()

,date,location,sales
0,2020-04-07,Delhi,3147
1,2020-01-03,Kolkata,9791
2,2020-05-25,Kolkata,3021
3,2020-04-29,Delhi,6455
4,2020-03-15,Delhi,7793


In [21]:
data.to_csv('Sales.csv')

In [23]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Spark Basic Operations").getOrCreate()

In [12]:
spark

In [24]:
from pyspark.sql import Row
from pyspark.sql.types import StructField, StructType, StringType, LongType

In [26]:
# DataFrame from csv
df = spark.read.csv("Sales.csv", inferSchema=True, header=True)

In [27]:
df.show()

+---+----------+--------+-----+
|_c0|      date|location|sales|
+---+----------+--------+-----+
|  0|2020-04-07|   Delhi| 3147|
|  1|2020-01-03| Kolkata| 9791|
|  2|2020-05-25| Kolkata| 3021|
|  3|2020-04-29|   Delhi| 6455|
|  4|2020-03-15|   Delhi| 7793|
|  5|2020-04-17| Kolkata| 6144|
|  6|2020-02-26|   Delhi| 3499|
|  7|2020-05-19|   Delhi| 8147|
|  8|2020-04-01|  Mumbai| 6955|
|  9|2020-05-15|Banglore| 5155|
| 10|2020-01-09|Banglore| 8911|
| 11|2020-05-09|  Mumbai| 9460|
| 12|2020-05-07|  Mumbai| 5116|
| 13|2020-04-29|  Mumbai| 2673|
| 14|2020-05-26| Kolkata| 3134|
| 15|2020-03-21|Banglore| 8718|
| 16|2020-02-29|  Mumbai| 7084|
| 17|2020-04-14| Kolkata| 3029|
| 18|2020-02-24|  Mumbai| 6320|
| 19|2020-02-13|   Delhi| 8881|
+---+----------+--------+-----+
only showing top 20 rows



In [28]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- date: string (nullable = true)
 |-- location: string (nullable = true)
 |-- sales: integer (nullable = true)



In [29]:
new_df = df.selectExpr("avg(sales)")

In [30]:
new_df.show()

+----------+
|avg(sales)|
+----------+
|  5998.385|
+----------+



In [31]:
total_sales = df.selectExpr("sum(sales)")
total_sales.show()

+----------+
|sum(sales)|
+----------+
|  35990310|
+----------+



In [34]:
df = df.drop('_c0')

In [35]:
df.groupBy('location').sum().show()

+--------+----------+
|location|sum(sales)|
+--------+----------+
|  Mumbai|   8662456|
| Kolkata|   9019388|
|   Delhi|   9194983|
|Banglore|   9113483|
+--------+----------+



In [40]:
from pyspark.sql.functions import year,month
from pyspark.sql.functions import to_date

df1 = df.withColumn('Month',month(df.date))
df1.show()

+----------+--------+-----+-----+
|      date|location|sales|Month|
+----------+--------+-----+-----+
|2020-04-07|   Delhi| 3147|    4|
|2020-01-03| Kolkata| 9791|    1|
|2020-05-25| Kolkata| 3021|    5|
|2020-04-29|   Delhi| 6455|    4|
|2020-03-15|   Delhi| 7793|    3|
|2020-04-17| Kolkata| 6144|    4|
|2020-02-26|   Delhi| 3499|    2|
|2020-05-19|   Delhi| 8147|    5|
|2020-04-01|  Mumbai| 6955|    4|
|2020-05-15|Banglore| 5155|    5|
|2020-01-09|Banglore| 8911|    1|
|2020-05-09|  Mumbai| 9460|    5|
|2020-05-07|  Mumbai| 5116|    5|
|2020-04-29|  Mumbai| 2673|    4|
|2020-05-26| Kolkata| 3134|    5|
|2020-03-21|Banglore| 8718|    3|
|2020-02-29|  Mumbai| 7084|    2|
|2020-04-14| Kolkata| 3029|    4|
|2020-02-24|  Mumbai| 6320|    2|
|2020-02-13|   Delhi| 8881|    2|
+----------+--------+-----+-----+
only showing top 20 rows



In [43]:
#Sales by Month
df2= df1.groupby('Month').sum()

In [44]:
df2.columns

['Month', 'sum(sales)', 'sum(Month)']

In [45]:
#SALES ACCORDING TO MONTH
df2.select(['Month','sum(sales)']).show()

+-----+----------+
|Month|sum(sales)|
+-----+----------+
|    1|   7215318|
|    3|   7320518|
|    5|   7156229|
|    4|   7175476|
|    2|   7122769|
+-----+----------+



In [48]:
#SALES ACCORDING TO REGION AND MONTH
df3 = df1.groupby(['Month','location']).sum().select(['Month','location','sum(sales)']).show()

+-----+--------+----------+
|Month|location|sum(sales)|
+-----+--------+----------+
|    5|  Mumbai|   1665824|
|    2|   Delhi|   1759888|
|    2|  Mumbai|   1843644|
|    3| Kolkata|   1786284|
|    5|Banglore|   1767963|
|    1|Banglore|   1840764|
|    2|Banglore|   1789870|
|    1|  Mumbai|   1736217|
|    4|Banglore|   1865797|
|    1| Kolkata|   1844824|
|    3|Banglore|   1849089|
|    4| Kolkata|   1882405|
|    2| Kolkata|   1729367|
|    1|   Delhi|   1793513|
|    5|   Delhi|   1945934|
|    3|  Mumbai|   1860438|
|    4|   Delhi|   1870941|
|    5| Kolkata|   1776508|
|    3|   Delhi|   1824707|
|    4|  Mumbai|   1556333|
+-----+--------+----------+

